# WASDE Data Exploration


1. Libraries and setup

In [5]:
# -- Setup environment and imports --

# Add 'src' directory to path to import project modules
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent / "src"))

# Load environment variables securely (e.g., WASDE_JWT)
from dotenv import load_dotenv
import os
load_dotenv()
my_token = os.getenv("WASDE_JWT")

# Core imports
from datetime import datetime
import pandas as pd

# Project configuration and functions
from config import RAW_DATA, PROCESSED_DATA
from wasde_downloader import fetch_wasde_releases, download_release_files, process_all_wasde_files, classify_crop_stage


2. Identifying reports available and download of the csv/xml files

In [2]:
# Step 1: Fetch WASDE releases metadata using your secure token
releases = fetch_wasde_releases(token=my_token)
print(f"{len(releases)} releases found.")

# Step 2: Download the XML files (limit optional for testing)
download_release_files(releases, limit=10)


316 releases found.


3. Extracting supply demand data of the files: Wheat, corn, soybean, soybean meal and soybean oil

In [3]:
# Step 3: Process all downloaded WASDE files and load into a DataFrame
df_wasde = process_all_wasde_files()
df_wasde.head()

181 XML files found in C:\Users\d1eg0\Desktop\python\Prediction\wasde\data\wasde_files
178 unique report dates found.
Processing 2010-07-09_wasde-07-09-2010.xml...
 → 214 rows extracted.
Processing 2010-08-12_wasde-08-12-2010.xml...
 → 214 rows extracted.
Processing 2010-09-10_wasde-09-10-2010.xml...
 → 214 rows extracted.
Processing 2010-10-08_wasde-10-08-2010.xml...
 → 214 rows extracted.
Processing 2010-11-09_wasde-11-09-2010.xml...
 → 214 rows extracted.
Processing 2010-12-10_wasde-12-10-2010.xml...
 → 214 rows extracted.
Processing 2011-01-12_wasde-01-12-2011.xml...
 → 214 rows extracted.
Processing 2011-02-09_wasde-02-09-2011.xml...
 → 214 rows extracted.
Processing 2011-03-10_wasde-03-10-2011.xml...
 → 214 rows extracted.
Processing 2011-04-08_wasde-04-08-2011.xml...
 → 214 rows extracted.
Processing 2011-05-11_wasde-05-11-2011.xml...
 → 214 rows extracted.
Processing 2011-06-09_wasde-06-09-2011.xml...
 → 214 rows extracted.
Processing 2011-07-12_wasde-07-12-2011.xml...
 → 214 r

,report_date,commodity,country,marketing_year,crop_stage,Beginning Stocks,Production,Imports,Domestic Total 2,Exports,Ending Stocks,Domestic Crush,Domestic Feed
0,2010-07-09,wheat,World 3/,2008/09,Current year,124.42,683.26,136.86,642.52,143.42,165.16,NaN,NaN
1,2010-07-09,wheat,United States,2008/09,Current year,8.32,68.02,3.46,34.29,27.64,17.87,NaN,NaN
2,2010-07-09,wheat,Total Foreign,2008/09,Current year,116.09,615.25,133.40,608.23,115.78,147.29,NaN,NaN
3,2010-07-09,wheat,Major Exporters 4/,2008/09,Current year,22.75,211.25,8.26,147.70,65.64,28.91,NaN,NaN
4,2010-07-09,wheat,Argentina,2008/09,Current year,2.35,10.10,0.03,5.33,6.77,0.39,NaN,NaN


4. Classifying the crop

In [6]:

    
if {"report_date", "marketing_year"}.issubset(df_wasde.columns):
    df_wasde["crop_stage"] = df_wasde.apply(
        lambda row: classify_crop_stage(str(row["marketing_year"]), str(row["report_date"])),
        axis=1
    )


5. Checking dataframe info

In [7]:
print(df_wasde.head())
print("*"*60)
print(df_wasde.info())


  report_date commodity              country marketing_year    crop_stage  \
0  2010-07-09     wheat            World  3/        2008/09  Current year   
1  2010-07-09     wheat        United States        2008/09  Current year   
2  2010-07-09     wheat        Total Foreign        2008/09  Current year   
3  2010-07-09     wheat  Major Exporters  4/        2008/09  Current year   
4  2010-07-09     wheat            Argentina        2008/09  Current year   

   Beginning Stocks  Production  Imports  Domestic Total 2  Exports  \
0            124.42      683.26   136.86            642.52   143.42   
1              8.32       68.02     3.46             34.29    27.64   
2            116.09      615.25   133.40            608.23   115.78   
3             22.75      211.25     8.26            147.70    65.64   
4              2.35       10.10     0.03              5.33     6.77   

   Ending Stocks  Domestic Crush  Domestic Feed  
0         165.16             NaN            NaN  
1         

6. Exporting to csv to make the PowerBi visualization

In [8]:
df_wasde.to_csv(PROCESSED_DATA / "wasde_timeseries_clean.csv", index=False)
